In [1]:
!pip install flask

In [2]:
!pip install --upgrade flask watchdog

In [2]:
# Import necessary libraries
from flask import Flask, request, render_template
import numpy as np
import pickle

COMPLAINT_DICT = {'Chest pain, Transfer': 44725, 'Confusion, s/p Fall': 19980, 'R Leg pain, R Leg swelling': 14455, 'L Arm numbness, L Arm swelling': 7728, 'Epigastric pain': 4860, 'Coffee ground emesis': 4250, 'L Leg pain, s/p Fall': 1824, 'Abd pain, N/V': 1644, 'R Foot pain': 1200, 'Hematemesis': 1190, 'Chest pain, Nausea': 1188, 'Chest pain, N/V': 1152, 'Abnormal labs': 1047, 'Dyspnea': 784, 'Diarrhea, Hypotension': 690, 'Fatigue, s/p Fall': 660, 'R RIB PAIN': 576, 'LOWER EXTREMITY PAIN': 560, 'RLQ abdominal pain': 540, 'Right sided abdominal pain': 540, 'Abd pain, Right sided abdominal pain': 504, 'Abnormal labs, Hyperglycemia': 504, 'Tachycardia': 484, 'Abd pain, Diarrhea, Vomiting': 480, 'R FOOT ULCER/CELLULITIS': 350, 'Abd pain': 270, 'Dehydration, Nausea, Rash': 252, 'Psych eval': 224, 'Back pain, Decreased PO intake, R Ear pain': 210, 'DISLODGED ABD TUBE': 208, 'R Leg pain, Weakness': 192, 'Presyncope, Weakness': 192, 'Insomnia': 144, 'RIGHT FOOT INFECTION': 144, 'L Arm pain, L Arm swelling': 138, 'N/V, Tachycardia': 120, 'ABDOMINAL MASS, FEVER': 112, 'N/V, Hyperglycemia': 96, 'R FOOT PAIN': 80, 'N/V, Tinnitus': 72, 'Weakness, Atrial fibrillation, Transfer': 64, 'R LEG CELLULITIS': 48, 'Psychiatric hold, Altered mental status': 48, 'ABD PAIN': 30, 'Chest pain': 18, 'Neck pain, Med refill': 15, 'PE': 10, 'PICC LINE INFECTION': 6}
DISPOSITION_DICT = {'HOME': 91265, 'ADMITTED': 22909, 'TRANSFER': 434}

# Load the trained model
model = pickle.load(open("C:\\Users\\Karan\\Desktop\\Patient_Wait_time\\model.pkl", "rb"))

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Retrieve input data from the form
        chiefcomplaint = request.form['chiefcomplaint']
        dbp_triage = float(request.form['dbp_triage'])
        pain_triage = float(request.form['pain_triage'])
        sbp_triage = float(request.form['sbp_triage'])
        temp_triage = float(request.form['temp_triage'])
        ndc = float(request.form['ndc'])
        o2sat_triage = float(request.form['o2sat_triage'])
        disposition = request.form['disposition']
        race = request.form['race']
        
        if chiefcomplaint in COMPLAINT_DICT.keys():
            chiefcomplaint_encoded = COMPLAINT_DICT.get(chiefcomplaint)
        else:
            chiefcomplaint_encoded = 1

        if disposition in DISPOSITION_DICT.keys():
            disposition_encoded = DISPOSITION_DICT.get(disposition)
        else:
            disposition_encoded = 1

        # Assemble input features in the order expected by the model
        features = np.array([[chiefcomplaint_encoded, dbp_triage, pain_triage, sbp_triage, temp_triage, ndc, disposition_encoded]])
        prediction = model.predict(features)
        
        # Render prediction template with the predicted wait time
        return render_template("index.html", prediction_text=f"The wait time is Approximately {int(prediction[0])} Minutes")
    except Exception as e:
        # On error, render the error template with a descriptive message
        return render_template('error.html', message=f"An error occurred while processing the prediction: {e}")

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Apr/2024 22:36:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2024 22:36:33] "GET /favicon.ico HTTP/1.1" 404 -
c:\Users\Karan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
127.0.0.1 - - [19/Apr/2024 22:36:55] "POST /predict HTTP/1.1" 200 -
